In [1]:
# Ноубук для парсинга данных

In [6]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
from concurrent.futures import ThreadPoolExecutor
from fake_useragent import UserAgent
import numpy as np
from datetime import datetime
import json
import sys
import keyboard
import time
pd.options.display.max_columns = None

In [4]:
import numpy as np

In [1]:
import pandas as pd

In [11]:
pd.options.display.max_columns = None

In [17]:
train = pd.read_csv('autoru_cars_dataset.csv')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,9,11,14,16,19,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
train.head(2)

,Unnamed: 0,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,acceleration,fuel_rate
0,0,внедорожник 5 дв.,VOLVO,https://auto.ru/cars/used/sale/volvo/xc90/1101...,синий,{'id': '0'},Продажа а/м осуществляется ОФИЦИАЛЬНЫМ ДИЛЕРОМ...,2.0,235.0,"{'esp': True, 'airbag-driver': True, 'aux': Tr...",дизель,//avatars.mds.yandex.net/get-autoru-vos/209286...,85452.0,2014.0,"{'code': 'XC90', 'name': 'XC90', 'ru_name': 'X...",XC90,2.0d AT (235 л.с.) 4WD,5.0,1.604647e+12,RUR,2016.0,1101191623-464cf4be,"{'id': '20228874', 'name': 'II', 'ru_name': '2...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2799000.0,7.8,5.7
1,1,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s80/11013...,чёрный,{'id': '0'},При покупке автомобиля до 15 ноября \n► Выгод...,2.5,249.0,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,//avatars.mds.yandex.net/get-autoru-vos/216441...,116410.0,2009.0,"{'code': 'S80', 'name': 'S80', 'ru_name': 'S80...",S80,2.5 AT (249 л.с.),4.0,1.604660e+12,RUR,2013.0,1101345022-3e0e4f8c,"{'id': '7974165', 'name': 'II Рестайлинг', 'ru...",SEDAN AUTOMATIC 2.5,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1277000.0,7.0,8.6


In [20]:
set(train.columns)-set(test.columns)

{'Unnamed: 0', 'acceleration', 'fuel_rate', 'price'}

In [19]:
test.head(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [3]:
def get_html_text(url):

    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    return response.text

def get_car_urls(get_html_text):
    urls_lenta = bs(get_html_text,'html.parser').find_all('a',class_='Link ListingItemTitle-module__link')
    return urls_lenta

def get_all_links_pages(sort_list,brand_list,n):
    new_list = []
    urls_list = []
    for sort in sort_list:
        for brand in brand_list:
            for index in range(1,n+1):
                new_list.append((sort,brand,index))
    
    for tuple_params in new_list:
        url = 'https://auto.ru/rossiya/cars/'+tuple_params[1]+'/all/?sort='+tuple_params[0]+'&page='+str(tuple_params[2])
        urls_list.append(url)
        
    return urls_list

def make_all(url):
    html_text = get_html_text(url)
    car_urls = get_car_urls(html_text)
    car_urls = [link['href'] for link in car_urls]
    list_cars.append(car_urls)
    if len(list_cars)%50==0:
        print(len(list_cars))

def find_float_int(s):
    list_num = s[s.find('Разгон'):len(s)].replace('Разгон','').replace('Расход','')\
                                        .replace(' л','').replace('с','').split(' ')
    list_num = [float(x) for x in list_num]
    return list_num,s

def get_tech(url):
    try:
        start_time = time.time()
        parsing_unixtime = datetime.now().timestamp()
        html = get_html_text(url)
        page = bs(html,'html.parser')
        pattern_mileage = re.compile('\d+')
        pattern_fuel = re.compile('\d+\.\d+')

        try:
            productionDate = page.find('a',class_='Link Link_color_black').text
        except: productionDate = np.nan
        
        try:
            bodyType = page.find('li',class_='CardInfoRow CardInfoRow_bodytype').a.text.encode('ISO-8859-1').decode('UTF-8')
        except: bodyType = np.nan
        
        try:
            color = page.find('li',class_='CardInfoRow CardInfoRow_color').a.text.encode('ISO-8859-1').decode('UTF-8')
        except: color = np.nan
    
        description = page.find('div',class_="CardDescription__textInner")
        if description!=None:
            description = description.text.encode('ISO-8859-1').decode('UTF-8')
        else:
            description=np.nan
            
        try:    
            engine_list = page.find('li',class_="CardInfoRow CardInfoRow_engine").div.text.encode('ISO-8859-1')\
                                    .decode('UTF-8').replace(' ','').split('/')
            engineDisplacement = float(engine_list[0].replace('л',''))

            enginePower = float(engine_list[1].replace('\xa0л.с.',''))
            
            fuelType = engine_list[2]
        except:
            try:
                engine_list = page.find('li',class_="CardInfoRow CardInfoRow_engine").text.encode('ISO-8859-1')\
                                    .decode('UTF-8').replace(' ','').split('/')
                engineDisplacement = np.nan
            
                enginePower = np.nan
            
                fuelType = np.nan
            except:
                engine_list, engineDisplacement, enginePower, fuelType = np.nan, np.nan, np.nan, np.nan

        try:
            mileage = page.find('li',class_='CardInfoRow CardInfoRow_kmAge').text.encode('ISO-8859-1').decode('UTF-8')
            mileage = float(pattern_mileage.findall(mileage)[0]+pattern_mileage.findall(mileage)[1])
        except: mileage = np.nan
            
        try:
            vehicleTransmission = page.find('li',class_='CardInfoRow CardInfoRow_transmission')\
                                        .text.encode('ISO-8859-1').decode('UTF-8').replace('Коробка','')
        except: vehicleTransmission = np.nan
        
        try:
            wheel_drive = page.find('li',class_='CardInfoRow CardInfoRow_drive')\
                                        .text.encode('ISO-8859-1').decode('UTF-8').replace('Привод','')
        except: wheel_drive = np.nan
        
        try:
            rudder = page.find('li',class_='CardInfoRow CardInfoRow_wheel')\
                                        .text.encode('ISO-8859-1').decode('UTF-8').replace('Руль','')
        except: rudder = np.nan
    
        try:
            state = page.find('li',class_='CardInfoRow CardInfoRow_state')\
                                        .text.encode('ISO-8859-1').decode('UTF-8').replace('Состояние','')
        except: state = np.nan
        
        owners = page.find('li',class_='CardInfoRow CardInfoRow_ownersCount')
        if owners!=None:
            owners = owners.text.encode('ISO-8859-1').decode('UTF-8').replace('Владельцы','')
        else: owners=np.nan
            
            
        if page.find('li',class_='CardInfoRow CardInfoRow_owningTime')!=None:
            owner_time = page.find('li',class_='CardInfoRow CardInfoRow_owningTime')\
                                        .text.encode('ISO-8859-1').decode('UTF-8').replace('Владение','')
        else: owner_time = np.nan

            
        pts = page.find('li',class_='CardInfoRow CardInfoRow_pts')
        if pts!=None:
            pts = pts.text.encode('ISO-8859-1').decode('UTF-8').replace('ПТС','')
        else: pts = np.nan

        try:    
            customs = page.find('li',class_='CardInfoRow CardInfoRow_customs')\
                                            .text.encode('ISO-8859-1').decode('UTF-8').replace('Таможня','')
        except:customs=np.nan
        
        try:
            url_char = page.find('a',class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')['href']
            page_char = bs(get_html_text(url_char),'html.parser')
        except:page_char = np.nan
        
        try:
            fuel = page_char.find_all('dl',class_='list-values list-values_view_ext clearfix')[1]\
                                                                .text.encode('ISO-8859-1').decode('UTF-8')
        except:fuel = np.nan

        try:    
            clearance_min = float(page_char.find('div',class_='catalog__deminsion catalog__deminsion_right').text)
        except:clearance_min = np.nan
        
        try:
            clearfix = page_char.find('dl',class_='list-values clearfix').text.encode('ISO-8859-1').decode('UTF-8')
        except:clerfix=np.nan
        
        try:
            title = page_char.find('h1',class_='catalog__title sidebar2__block').text.replace(' ','').encode('ISO-8859-1').decode('UTF-8')
        except:title=np.nan
        
        price = page.find('span',class_='OfferPriceCaption__price')
        if price!=None:
            price = page.find('span',class_='OfferPriceCaption__price').text.encode('ISO-8859-1').decode('UTF-8')
            price = pattern_mileage.findall(price)
        else: price=np.nan
            
        technical_dict = {'car_url':url,
                    'price':price,
                    'productionDate':productionDate,
                    'bodyType':bodyType,
                    'color':color,
                    'description':description,
                    'engine_list':engine_list,
                    'engineDisplacement':engineDisplacement,
                    'enginePower':enginePower,
                    'fuelType':fuelType,
                    'mileage':mileage,
                    'vehicleTransmission':vehicleTransmission,
                    'wheel_drive':wheel_drive,
                    'rudder':rudder,
                    'state':state,
                    'owners':owners,
                    'owner_time':owner_time,
                    'pts':pts,
                    'customs':customs,
                    'fuel':fuel,
                    'clearfix':clearfix,
                    'title':title,
                    'parsing_unixtime':parsing_unixtime
                    }
        
        main_data.append(technical_dict)
        if len(main_data)%100==0:
            print(len(main_data))
            json_dump(main_data)
            
    except: 
        main_data.append(np.nan)
def json_dump(main_data):
    if len(main_data)!=0:
        with open('main_data.json', 'w', encoding='utf-8') as f:
            json.dump(main_data, f, ensure_ascii=False, indent=4)

In [4]:
sort_list = ['fresh_relevance_1-desc','cr_date-desc','price-desc',
             'year-desc','year-asc','km_age-asc','alphabet-asc','autoru_exclusive-desc',
            'price_profitability-desc','proven_owner-desc']

brand_list = test.brand.unique()

In [5]:
links_pages = get_all_links_pages(sort_list,brand_list,n=99)

In [6]:
%%time
list_cars = []

with ThreadPoolExecutor(14) as executor:
    executor.map(make_all, links_pages[:2])

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\User\anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Wall time: 3.37 s


In [8]:
c=0
for i in list_cars:
    for k in i:
        c+=1
print('Всего машин:',c)

Всего машин: 76


In [14]:
car_urls = []
for list_car in list_cars:
    for url in list_car:
        if 'https://auto.ru/cars/used/' in url:
            car_urls.append(url)

In [10]:
main_data = []
with ThreadPoolExecutor(14) as executor:   
    for r in executor.map(get_tech,car_urls):
        try:  
            if keyboard.is_pressed('q'):
                print('Вы вышли из программы')
                sys.exit()
            else:
                pass
        except:
            break

In [19]:
for key in main_data:
    print(key,'\n')

{'car_url': 'https://auto.ru/cars/used/sale/skoda/octavia/1103237201-3ad0ce6c/?geo_id=225', 'price': ['375', '000'], 'productionDate': '2009', 'bodyType': 'лифтбек', 'color': 'синий', 'description': 'Машина без капиталовложений, без серьёзных ДТП, экономичный двигатель, все работает, ручник, свет, печка, поворотники, замена масла и фильтров 1500км назад, новые колодки и задние диски. В Коробке поменяно сцепление, шток и выжимной и масло Likvi molli, ЛКП в хорошем состоянии, сигнализация цз, мрз, масло от замены до замены без долива, торг у капота. Свободное место в ПТС есть. ', 'engine_list': ['1.6л', '102\xa0л.с.', 'Бензин'], 'engineDisplacement': 1.6, 'enginePower': 102.0, 'fuelType': 'Бензин', 'mileage': 142000.0, 'vehicleTransmission': 'механическая', 'wheel_drive': 'передний', 'rudder': 'Левый', 'state': 'Не требует ремонта', 'owners': '3 или более', 'owner_time': nan, 'pts': 'Оригинал', 'customs': 'Растаможен', 'fuel': 'ТопливоАИ-95ПриводпереднийРазгон11.8 сРасход7.1 л', 'clearfi

In [10]:
with open('main_data.json', 'r', encoding='utf-8') as f: 
    main_data = json.load(f)

In [16]:
data_dict = {}
for i in main_data:
    if type(i)!=float:
        data_dict[i['car_url']] = i

In [17]:
len(set(data_dict.keys()))

117099

In [21]:
len(main_data)

120124